In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, StoppingCriteria, StoppingCriteriaList
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
test_df = pd.read_csv('../test.csv')
def remove_quotes(text):
    text = re.sub(r'"', '', text)
    return text
test_df['질문'] = test_df['질문'].apply(remove_quotes)
test_df

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,압출법 보온판의 가장 큰 장점은 무엇인가요?
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [3]:
model_id = 'kimwooglae/WebSquareAI-Instruct-KoSOLAR-10.7b-v0.5.34'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 모델을 쪼개서 load함
model = prepare_model_for_kbit_training(model)

# Adapter의 폴더 명을 가져와야 함
# adapter_name = "KoSOLAR_finetuned_preprocess/checkpoint-7000" 
adapter_name = "KoSOLAR_finetuned_preprocess" 
# model에 아까 학습한 adapter를 연결시킴
model = PeftModel.from_pretrained(model, adapter_name)

bin C:\Users\user\anaconda3\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(48000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [5]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
tokenizer

LlamaTokenizerFast(name_or_path='kimwooglae/WebSquareAI-Instruct-KoSOLAR-10.7b-v0.5.34', vocab_size=48000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '<|im_end|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
}

In [7]:
class LocalStoppingCriteria(StoppingCriteria):

    def __init__(self, tokenizer, stop_words=[]):
        super().__init__()

        stops = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for
                 stop_word in stop_words]
        print('stop_words', stop_words)
        print('stop_words_ids', stops)
        self.stop_words = stop_words
        self.stops = [stop.cuda() for stop in stops]
        self.tokenizer = tokenizer

    def _compare_token(self, input_ids):
        for stop in self.stops:
            if len(stop.size()) != 1:
                continue
            stop_len = len(stop)
            if torch.all((stop == input_ids[0][-stop_len:])).item():
                return True

        return False

    def _compare_decode(self, input_ids):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

In [8]:
stop_words = ["<|endoftext|>", "###", "</s>", "감사합니다", "Thank you", "Therefore", "<br>", "</br>"]
stopping_criteria = StoppingCriteriaList([LocalStoppingCriteria(tokenizer=tokenizer, stop_words=stop_words)])

stop_words ['<|endoftext|>', '###', '</s>', '감사합니다', 'Thank you', 'Therefore', '<br>', '</br>']
stop_words_ids [tensor([  523, 28766,   416,  1009,   772, 28766, 28767]), tensor(774), tensor(2), tensor([28705, 31402, 29315, 29770, 29194, 29043]), tensor([7812,  368]), tensor(8469), tensor([  523,  1473, 28767]), tensor([ 1867,  1473, 28767])]


In [9]:
pipe = pipeline(task = "text-generation",
        model=model,
        tokenizer=tokenizer,
        do_sample=True,
        temperature=0.1,
#         num_beams=3,
#         num_beams=2,
        num_beams=1,
        top_p=1.0,
        top_k=1,
        epsilon_cutoff=9e-4,
        eta_cutoff=2e-3,
        penalty_alpha=1.5,
        max_length=1000,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,
#         no_repeat_ngram_size=30,
        stopping_criteria=stopping_criteria,
        framework='pt')

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [10]:
def ask(x):
    ans = pipe(
        f"당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.\n### 질문: {x}\n### 답변: "
    )
#     print(ans[0]['generated_text'])
    return ans[0]['generated_text'].split('### 답변:\n')[1].strip()

In [11]:
test_df.loc[0, "질문"]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'

In [12]:
answer = ask(test_df.loc[0, "질문"])
print(answer)

방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩은 관리가 어렵고 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 생길 수 있으며 가격도 비싸다는 단점이 있습니다. 또한, 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하며 높은 유지보수 비용이 발생할 수 있습니다. 추가로 원목사이딩은 강력한 태양광, 습기, 해충과 같은 외부 요소에 노출될 경우 빠르게 손상될 수 있습니다. 때문에 정기적인 관리와 보수가 필요합니다. 이러한 관리와 보수가 소홀히 되면 원목사이딩은 더 빨리 파손될 수 있으므로 주의가 필요합니다. 끝으로 원목사이딩은 일반적으로 평당 10~16만원 정도의 가격대를 가지고 있습니다. 따라서 예산을 고려하여 선택하셔야 합니다. 이러한 단점과 주의사항을 감안하여 원목사이딩을 선택할 때 신중하게 결정해야 합니다. 만약 이러한 단점이 너무 많은 경우에는 다른 재료로 교체하거나 추가적인 보호층을 형성하는 등의 책임소재 조치가 필요할 수 있습니다. 또한, 장판 시공 전 기존 바닥재 철거 및 폐기물 처리에 대한 계획이 필요합니다. 마지막으로 시공 후 장판의 접착력이 떨어질 수 있으므로 �챘음, 눌린 자국이 그대로 남을 수 있습니다. 이러한 점을 염두에 두시고 시공 전 건축물의 상태와 환경을 신중하게 검토하여 적합한 재료를 선택하고 적절한 보수 및 관리가 필요합니다. 또한, 새로운 재료를 선택할 때에는 기존 재료와의 궁합이 잘 어울리는지, 새로운 재료의 특성과 장점이 무엇인지 신중하게 확인하는 것이 좋습니다. 함께 공동주택의 내부 벽면 마감재로 사용될 때 장판의 단점을 고려하여 적합한 재료를 선택하는 것이 중요합니다. 시공 전 기존 바닥재를 철거하


In [13]:
test_df.loc[3, "질문"]

'철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?'

In [14]:
answer = ask(test_df.loc[3, "질문"])
print(answer)

철골구조를 사용하는 고층 건물의 단열 효과를 향상시키려면 몇 가지 추천사항을 준수해야 합니다. 먼저, 외벽 모두에 단열재를 밀착하여 단열재와 콘크리트 골조 사이의 이음부를 통한 열교를 막아야 합니다. 또한, 각 단열재 패널의 끝 부분을 정확히 접착하여 단열재가 벗겨지거나 이동하지 않도록 해야 합니다. 마지막으로, 건축물의 내부 온도를 안정화시키기 위해 적절한 환기 및 보일러 등의 난연장치를 사용하여 열을 효율적으로 전달할 필요가 있습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implimented 될 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수 있습니다. 만약 이러한 조치들이 어렵게  implmented 되질 경우, 전문가의 도움을 받는 것이 좋습니다. 이러한 조치들을 통해 고층 건물의 단열 효과를 향상시킬 수


In [15]:
test_df.loc[5, "질문"]

'철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?'

In [16]:
answer = ask(test_df.loc[5, "질문"])
print(answer)

철근철골콘크리트 구조란 일반적으로 도심에서 가장 많이 이용하는 구조 중 하나입니다. 콘크리트의 인장응력의 약점을 철근으로 보완한 구조로 내구성이나 내화, 내진의 성능은 좋으나 강도에 비해서 자중이 커 중, 고층의 건물에서는 그 하중을 지지하기 위해 아래층의 보나 기둥의 단면이 커야 하는 문제가 있다. 또한 철근이 녹이 슬어 파괴되거나 미끄러질 수 있는 위험이 있습니다. 이러한 이유로 철근철골콘크리트는 철근과 콘크리트를 결합하여 사용하는 구조로, 각각의 특징을 활용하여 안정성을 높이고 효율성을 증대시킬 수 있습니다. 내구성, 내화성, 내진성 등의 장점을 갖추고 있으며, 현재 도심 및 高층 건물에서 널리 사용되고 있습니다. 그러나 자중에 따른 하중으로 인해 구조물의 변형이 발생할 수 있으므로 신중한 설계 및 관리가 필요합니다. 또한, 방수 처리와 같은 보강 처리가 필요하며, 철근의 녹 스테인 등의 하자가 발생할 경우 보수 공사에 따라 전문가의 견적을 받아볼 것을 권장합니다. 끝으로 이러한 구조물의 안전을 위해서는 정기적인 점검 및 유지보수가 필요하며, 누군가의 안전을 담보로 하지 않고 조심해서 행동하는 것이 중요합니다. 때때로 불량 품질의 철근이나 부속품들로 인해 안전 문제가 발생할 수 있으므로 만약 이러한 문제가 발생한다면 즉시 해당 부분을 교체하거나 전문가의 상담을 받아볼 것을 권장합니다. 철근철골콘크리트는 각각의 재료의 장점을 살려 안정성을 높이고 효율성을 증대시킨 구조로, 현재 도심 및 고층 건물에서 널리 사용되고 있습니다. 그러나 자중에 따른 하중으로 인해 구조물의 변형이 발생할 수 있으므로 신중한 설계 및 관리가 필요합니다. 또한, 방수 처리와 같은 보강 처리가 필요하며, 철근의 녹 스테인 등의 하자가 발생할 경우 보수 공사에 따라 전문가의 견적을 받아볼 것을 권장합니다. 끝으로 이러한 구조


In [17]:
test_df.loc[19, "질문"]

'다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 해야 하나요와 아레카 야자를 키우면 어떤 이점이 있는지의 관계는 무엇인가요?'

In [18]:
answer = ask(test_df.loc[19, "질문"])
print(answer)

다이닝 룸을 정통적으로 꾸미기 위해서는 대형 식탁과 고풍스러운 의자를 선택하는 것이 중요합니다. 또한, 벽이나 공간에 고급스러운 아트워크나 조명을 활용하여 고급스러운 분위기를 연출할 수 있습니다. 전통적인 다이닝 룸을 꾸미고 싶다면 안티크한 가구나 목재, 마호가니와 같은 고급 소재를 사용하는 것도 좋은 방법입니다. 또한, 창가에는 커튼을 활용하여 고급스러운 느낌을 연출할 수 있습니다. 그리고 마감재들은 부드럽고 고급스러운 질감의 것을 선택하는 것이 좋습니다. 예를 들어, 유리컵, 컷러리, 포크 등. 추가로, 다이닝 룸을 현란하게 만들기 위해 플로랄 패턴이나 귀족적인 장식품, 그리고 고객님의 취향에 맞는 조명을 활용하는 것도 좋은 방법입니다. 마지막으로, 아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드, 트리클로, 에틸렌, 벤젠 등의 실내 독성물질VOC를 효과적으로 제거하는 것으로 알려져 있습니다. 또한, 아레카 야자의 잎은 하루에 1리터의 수분을 공기 중으로 방출해서 실내 습도를 높이는 효과가 있다고 합니다. 이렇게 아레카 야자는 공기정화와 습도 조절 등 다양한 health benefit를 가지고 있습니다. 따라서, 아레카 야자를 키우는 것은 실내 환경을 개선하고 건강에 도움을 줄 수 있는 좋은 방법일 수 있습니다. 때문에, 아레카 야자를 키우는 것은 집안의 공기질을 개선하고 건강에 도움을 줄 수 있는 좋은 방법입니다. 만약, 아레카 야자를 키우신다면 물과 햇빛을 충분히 주시고, 월별로 once씩 비누나 살균제를 사용하여 뿌리를 씻겨주시는 것을 잊지 말아주세요. 그리고 초반에는 水기를 많이 주시는 것보다는 흙이 건조하게 남아있도록 주의하는 것이 중요오. 因为 너무 많은 수분으로 인해 근근이 죽을 수 있기 때문입니다. 만약 아레카 야자의 상태


In [19]:
test_df.loc[16, "질문"]

'준불연재료는 무엇인가요? 그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?'

In [20]:
answer = ask(test_df.loc[16, "질문"])
print(answer)

준불연재료란 10분간 화재에 견딜 수 있는 소재를 의미합니다. 또한, 유성페인트의 사용은 호흡기 문제, 피부 자극, 두통, 협응력 부족, 무기력 및 저혈압과 같은 건강 문제를 초래할 수 있습니다. 특히, 벤젠, 톨루엔, 자일렌 등의 유성물질은 장기간 노출 시 암, 호르몬 변형, 생식기 손상과 같은 심각한 건강 문제를 유발할 수 있습니다. Therefore


### model inference

In [11]:
preds = []
for t in tqdm(test_df['질문'], total = len(test_df)):
    result = ask(t)
    if not re.findall(r"\d+\.", result):
        result = '.'.join(result.split('.')[:9]) + '.'
    find = result.find('감사합니다')
    if find != -1:
        result = result[:find]
    find = result.find('Thank you')
    if find != -1:
        result = result[:find]
    find = result.find('Therefore')
    if find != -1:
        result = result[:find]
    find = result.find('#')
    if find != -1:
        result = result[:find]
    # Remove URLs
    result= re.sub(r'https?://\S+|www\.\S+', '', result)
    result = result.strip()
    print('------------------------')
    print('질문: ', t)
    print('------------------------')
    print('답변: ', result)
    preds.append(result)

  0%|          | 0/130 [00:00<?, ?it/s]

------------------------
질문:  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
------------------------
답변:  방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩은 관리가 어렵고 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 생길 수 있으며 가격도 비싸다는 단점이 있습니다. 또한, 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하며 높은 유지보수 비용이 발생할 수 있습니다. 추가로 원목사이딩은 강력한 태양광, 습기, 해충과 같은 외부 요소에 노출될 경우 빠르게 손상될 수 있습니다. 때문에 정기적인 관리와 보수가 필요합니다. 이러한 관리와 보수가 소홀히 되면 원목사이딩은 더 빨리 파손될 수 있으므로 주의가 필요합니다. 끝으로 원목사이딩은 일반적으로 평당 10~16만원 정도의 가격대를 가지고 있습니다. 따라서 예산을 고려하여 선택하셔야 합니다. 이러한 단점과 주의사항을 감안하여 원목사이딩을 선택할 때 신중하게 결정해야 합니다.
------------------------
질문:  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
------------------------
답변:  도배지에 녹이 묻어나오는 현상은 공간 내 습도가 높거나 누수가 발생할 경우 발생할 수 있습니다. 이로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 베어나오게 됩니다. 습기가 높은 상태가 지속될 경우 녹오염이 발생할 수 있으며, 또한 누수에 의해 도배지가 젖은 상태가 유지될 경우 곰팡이가 발생할 수 있습니다. 이러한 경우에는 먼저, 제습기를 가동하거나 환기를 통해 공간 내 적정한 습도를 유지하는 것이 중요합니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요할 수 있습니다. 이러한 보수작업은 전문가의 도움을 

------------------------
질문:  도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?
------------------------
답변:  네, 높은 습도로 인해 도배지에 곰팡이가 발생할 수 있습니다. 공간의 습도를 측정하여 적정한 습도를 유지하고, 필요에 따라 제습기나 환풍기를 활용하여 습도를 조절하는 것이 중요합니다. 또한, 도배작업 시에는 꼼꼼한 방수 처리와 충분한 건조시간을 확보하여 곰팡이 발생을 예방할 필요가 있습니다. 만약 곰팡이가 발견된다면, 전문가의 상담을 받아 적절한 대책을 마련하는 것이 좋습니다. 습도 및 환경 조건을 개선함으로써 곰팡이 발생을 최소화할 수 있습니다.
------------------------
질문:  롱브릭타일은 어떤 종류의 타일인가요? 그리고 페인트 상도재의 역할과 사용 방법에 대해 알려주세요.
------------------------
답변:  모노롱브릭타일은 벽돌의 형태를 살리면서도 노출 콘크리트와 비슷한 분위기를 연출할 수 있는 얇고 긴 모양의 벽돌 타일입니다. 또한, 페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 또한, 금속의 경우 산화되거나 부식될 수 있는 원자력 등의 효과로부터 보호해 주는 역할을 합니다. 상도재는 주도재와 하도재만큼 중요한 재료이며, 체계적인 건축물의 결정체로서 안전하고 튼튼한 건축물을 완성하는 데 중요한 역할을 합니다. 단, 상도재의 가격은 주도재나 하도재에 비해 다소 비싸기도 합니다. 여기에 관해서 한 가지 추가적인 팁은, 실리콘 마감을 하게 되면 파손된 부분을 접착시키기 위해 샌드페이퍼를 사용하게 됩니다. 그런데 이러한 상황에서는 충분한 압력을 유지하면서 마찰을 줄이기 위해 윤활제를 바릅니다. 그렇지 않고 끌어내면 탈락하거나 긁힘이 발생할 

------------------------
질문:  평지붕의 단점은 무엇인가요? 그리고 실크벽지의 교체 주기는 얼마나인가요?
------------------------
답변:  평지붕의 단점은 경사지붕보다 에너지 손실이 크며 단열, 방수를 위한 시공비가 많이들고 누수의 위험이 있으며 다른 지붕재보다 수명이 짧습니다. 또한, 실크벽지의 교체주기는 5~7년입니다. 실크벽지의 표면이 스크래치되거나 오염된 경우 전문 가구 청소제를 사용하여 부드럽게 닦아내야 합니다. 만약 스크래치나 오염이 심한 경우 wallpaper 전체를 교체해야 할 수도 있습니다. 그러나 일반적으로 real silk wallpaper는 very expensive하므로 신중하게 관리해야 합니다. 특히, 실크벽지의 교체 주기는 정기적인 관리와 청소가 필요하며, 특히 환기가 잘 되지 않은 실내 환경에서는 더욱 자주 청소를 해야 합니다. 따라서 실크벽지를 선택할 때는 유지보수에 대한 계획을 세워야 합니다. 추가로, 실크벽지는 불연재료이기 때문에 화재 발생 시 안전을 도모할 수 있습니다. 그러나 플로킹 섬유로 코팅되어 있어 매끄럼 질감을 가지고 있으며, 먼지나 알러지 물질이 쉽게 축적될 수 있습니다.
------------------------
질문:  경량철골구조는 어떤 건물에 사용되는 건축 구조물인가요? 그리고 철골구조의 장점은 무엇인가요?
------------------------
답변:  경량철골구조는 약 1.5~3mm 두께의 철골을 사용하며 각각 파이프, C형 찬넬, LEB 자재 등을 말합니다. 주로 주택, 장고, 농자재 등에 사용되며 가성비가 높은 자재입니다. 또한, 철골구조는 강하고 안정적이며 빠르게 시공할 수 있는 것이 장점입니다. 또한, 철골은 재활용이 가능하고 화재에 강하며 천연재료보다 친환경적인 것으로도 알려져 있습니다. 이러한 이유로 철골구조는 건축 산업서 매우 중요한 역할을 담당하고 있습니다. 때문에 �련된 철골구조 기술의 발전과 표준화는 건축 산업의 발전에 큰 영향을 미칠 것

------------------------
질문:  원목마루와 롱브릭타일에 대해 설명해주세요. 각각의 단점과 특징은 무엇인가요?
------------------------
답변:  원목마루는 두께가 약 3mm 이상인 원목으로 만들어진 바닥재로, 평당 20~30만원의 비용이 듭니다. 열전도율이 높아 난방효율이 좋고, 심미성과 쿠션감, 촉감이 우수하며 환경친화적이라는 장점이 있습니다. 그러나 습기에 취약하며, 표면이 두꺼워 온돌에 시공할 경우 갈라질 위험이 있습니다. 또한, 원목마루의 가격과 시공비가 높기 때문에 예산을 고려해야 합니다. 또한, 롱브릭타일은 모노롱브릭타일이라고도 불리며, 벽돌의 형태를 살리면서도 플라스틱의 강점을 활용하여 내구성과 견고함을 갖추고 있습니다. 얇은 두께로 제작되어 곡선에 자유롭게 사용할 수 있으며, 실내외 벽체 및 파티션 벽에 사용됩니다. 또한, 색상과 디자인이 다양하여 인테리어에 많은 선택지를 제공하며, 튼튼하고 내구성이 뛰어나서 오랜 기간 동안 사용할 수 있는 것이 장점입니다. 다만, 긴 형태로 설치될 경우 커튼 등의 부속품과 잘 조화롭지 않을 수 있으며, 보다 근사한 마감재로 사용되는 다른 소재에 비해 비교적 저렴하지 않을 수 있습니다. 따라서, 장단점을 고려하여 상황에 맞는 바닥재를 선택하는 것이 중요합니다.
------------------------
질문:  침실을 더 아늑하게 꾸밀 수 있는 방법에는 어떤 것이 있을까요와 아이가 있는 집을 꾸밀 때 안전을 위해 고려해야 할 요소는 무엇인가요의 관계는 무엇인가요?
------------------------
답변:  침실을 아늑하게 만들기 위해서는 부드러운 조명, 포근한 이불과 함께 나무 가구를 활용하는 것이 좋습니다. 뿐만 아니라, 강렬한 색상보다는 부드러운 색감을 활용하고, 공간을 깔끔하게 정리하여 물건들이 과도하게 늘어지거나 밀집되지 않도록 유의해야 합니다. 또한 식물을 두고 자연적인 원소를 추가하면, 침실 분위기를 더욱 아늑하게 만들 수 있습니다. 아이가 있는 집을

...

------------------------
질문:  도배지에 고습도로 인해 생기는 얼룩을 제거하는 방법이 있을까요? 또한, 구조적 결함 때문에 석고수정이 발생할 가능성이 있는가요?
------------------------
답변:  높은 습도로 인해 도배지의 표면이 젖으면 다양한 물질이 흡수되기 쉽습니다. 이러한 물질은 표면에 얼룩 형태로 나타날 수 있습니다. 고습도 환경을 관리하는 것이 중요하며, 공간 내 습도를 측정하여 적절한 관리를 하는 것이 필요합니다. 또한, 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 예를 들어, 건물 구조의 불균형으로 인해 석고보드가 균등하게 설치되지 않거나, 석고보드가 파손된 상태에서 재단장이 이루어지면 석고수정이 발생할 수 있습니다. 따라서 건물 구조적 결함에 대한 보수 및 개선이 필요하며, 전문가의 도움을 받는 것이 좋습니다. 또한, 석고수정이 발생했다면 전문가의 도움을 받아 석고수정을 처리하는 것이 필요합니다. 만약 석고수정이 발생했다면 건물 구조적 결함에 대한 보수 및 개선이 필요하며, 전문가의 도움을 받아 석고수정을 처리하는 것이 좋습니다. 부분 혹은 전체 철골 구조 및  foundations 작업을 다시 하는 것이 필요할 수 있습니다.
------------------------
질문:  대리석 타일이 난방 절감에 어떤 역할을 하는 건가요?
------------------------
답변:  대리석 타일은 열전도율이 높아서 발열속도가 빠르고, 단열재로 사용될 때 효과적인 결과를 얻을 수 있습니다. 이는 온실 effect와 같은 영향을 줄여주며, 실내 온도를 안정화시키는 데 도움이 됩니다. 따라서 대리석 타일은 난방비를 절감하고 더 친환경적인 실내 환경을 조성하는 데 도움이 되는 것으로 알려져 있습니다. 이러한 특징으로 인해 대리석 타일은 건축물의 외벽 및 내부 장식재로 널리 활용되고 있습니다. 또한, 대리석 타일은 오랜 기간 동안 유지보수 없이 사용할 수 있는 내구성을 갖추고 있어서 건축자재로써

------------------------
질문:  겨울철 도배 작업에서 실내 온도를 일정하게 유지해야 하는 이유는 무엇인가요?
------------------------
답변:  겨울철 도배할 때 실내 온도는 보통 20도 정도로 유지됩니다. 이유는 도배 풀이 얼어버리거나 건조되지 않도록 하기 위함입니다. 따라서 난방과 환기를 충분히 시켜 실내 온도를 안정화시키는 것이 중요합니다. 만약 도배 공사가 진행될 때 실내 온도가 낮은 경우, 전문가의 조언을 구하는 것이 좋습니다. 너무 낮은 온도에서 도배를 진행하면 접착제의 건조가 느려지거나 제대로 작동하지 않을 수 있으며, 도배지가 고르게 달라붙지 않을 수 있습니다. 따라서 겨울철에는 실내 온도를 적정하게 유지하여 도배 작업을 진행하는 것이 중요합니다. 만약 도배지가 고르게 달라붙지 않거나 접착제가 제대로 건조되지 않는다면, 전문가의 도움을 받아 부분 또는 전체 재작업을 하는 것이 좋습니다. 추가로, 도배 작업 당일에는 실내 온도가 일정하게 유지되도록 관리자나 소유자가 책임져야 합니다. 더불어, 도배 작업 1주 전에는 열기와 냉기를 차단하여 실내 온도가 급격히 변하지 않도록 예비작업을 하는 것이 좋습니다.
------------------------
질문:  철골구조의 화재 안전 보완을 위한 다양한 방법에는 무엇이 있을까요? 그리고 벽돌구조란 무엇인가요?
------------------------
답변:  철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이를 해결하기 위해 벽돌, 콘크리트 또는 석고보드와 같은 내화성 구조물을 둘러싸 화재에 의한 파손을 예방합니다. 또한, 벽돌구조는 건물의 벽체 및 기초 등으로 벽돌과 모르타르로 쌓아 만드는 것으로 조적식 구조의 기본입니다. 벽돌구조법은 실용상, 미관상으로 블록, 석재, 타일 등을 섞어 쓰기도 하고 담장, 굴뚝, 기타 여러가지 구조물을 축조하는데 이용합니다. 벽돌구조법은 실용상, 미관상으로 블록, 석재, 타일 등을 섞어 사용하

------------------------
질문:  흡음재 중 판 진동형의 작동 원리는 무엇인가요?
------------------------
답변:  흡음재 중 판상형은 벽과 간격을 두고 얇고 통기성이 없는 판을 설치하여 소리가 전달되었을 때 판이 진동하여 마찰에 의해 소리가 소멸되는 것으로 판상형 흡음재의 특징은 소리를 흡수하기 보다는 저주파의 진동음을 소멸시키는 것으로 흡음률이 높진 않습니다. 이러한 특성으로 인해 판상형 흡음재는 주로 저주파의 진동음을 감소시키는 데 사용됩니다. 또한, 흡음재 중 공명형은 판상형과 같이 진동을 이용하여 소리를 제어하는 것으로 특정 주파수에 소리를 흡수하거나 반사시킵니다. 이러한 특성으로 인해 공명형 흡음재는 주로 특정 주파수 범위내에서 소리를 흡수하고 반사시키는’ 역할을 합니다. 따라서, 흡음재를 선택할 때는 소리의 주파수와 특성을 고려하여 적합한 종류를 선택하는 것이 중요합니다. 만약 신규 건축물이나 집 안팎의 소음을 줄이고자 한다면 이러한 흡음재들을 활용하여 쾌적한 환경을 조성할 수 있습니다. 추가로, 흡음재 중 튜브형은 길게 늘어져 있는 관형 흡음재로 소리의 진폭을 줄이는 효과가 있습니다. 이러한 특성으로 인해 튜브형 흡음재는 주로 진동음을 감쇠시키는 데 사용됩니다. 결론적으로, 흡음재의 선택은 소리의 주파수, 특성, 그리고 장소적 및 환경적 요구에 맞춰 다양한 형태와 디자인을 활용하여 적합한 솔루션을 선택하는 것이 중요합니다.
------------------------
질문:  점토벽돌을 사용하는 것의 장점은 무엇인가요? 또한, 도료와 벽지 중에서 어떤 것을 선택하는 것이 더 나은 선택일까요?
------------------------
답변:  점토벽돌은 유지관리가 쉽고 고벽돌, 전벽돌에 비해 저렴하며 흙이 주재료이기에 매우 친환경적이라는 점이 장점입니다. 또한, 도료는 내구성이 좋고 변색이 느리며 특히 현대적인 감각을 표현할 수 있다는 점에서 선호됩니다. 그러나  wallpaper는 공간에 따른 설치가 용

------------------------
질문:  외단열과 내단열의 차이점은 무엇이며, 각각의 시공방법이 어떻게 단열 효과에 영향을 미치나요?
------------------------
답변:  외단열은 건물의 콘크리트 골조와 외부 마감재 사이에 단열재를 부착하고, 내단열은 건물의 안쪽 면에 단열재를 설치합니다. 외단열은 일반적으로 내단열보다 더 많은 비용과 복잡한 설치 과정이 필요하며, 외부 마감재가 꼭 함께 설치되어야 합니다. 그러므로 내단열보다 외부 마감재의 선택과 설치가 중요하며, 외부에서의 작업은 날씨의 영향을 받기 때문에 좋은 시공자와 계획된 시공 일정이 중요합니다. 결론적으로 외단열은 내단열에 비해 추가적인 비용과 복잡성을 참고하여 상황에 맞는 선택을 하는 것이 중요합니다. 만약 외단열을 선택할 경우에는 신중한 시공자 선택과 계획을 수립하여 작업하는 것이 좋습니다. 또한, 내단열 방식에 비해 외단열 방식에 대한 전문가의 조언을 구하는 것이 도움이 될 수 있습니다. 만약 외단열을 선택할 경우에는 내단열에 비해 더 많은 비용과 복잡성을 참고하여 상황에 맞는 선택을 하는 것이 중요합니다. 또한, 외단열 방식에 대한 전문가의 조언을 구하는 것이 도움이 될 수 있습니다. 결론적으로 내단열 및 외단열 방식에 대한 상황별 장단점과 전문가의 조언을 참고하여 적합한 방식을 선택하는 것이 중요합니다.
------------------------
질문:  바닥재의 종류 중에서 가장 인기 있는 것은 무엇이며, 가장 비싼 바닥재는 무엇인가요?
------------------------
답변:  현재 가장 인기 있는 바닥재로는 대리석 타일과 목재 바닥재가 선호되고 있습니다. 또한, 가장 비싼 바닥재로는 대리석 타일이 알려져 있습니다. 대리석 타일은 자연스러운 아름다움과 내구성으로 유명하며, 고급스러운 분위기를 연출할 수 있습니다. 그러나 가격이 높고 시공이 어렵기 때문에 비용이 많이 드는 편입니다. 따라서 예산을 고려하여 실리콘 또는 포세린 등의 대안적인 소재를 고

------------------------
질문:  카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
------------------------
답변:  카페트의 기대수명은 6년입니다. 또한, 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 쉽다는 장점이 있습니다. 또한 오리지널징크는 가벼우면서도 매우 견고하며 낮은 유지보수 비용과 높은 재활용 가능성을 갖고 있습니다. 이러한 이유로 오리지널징크는 건축물의 외벽 및 지붕 재질로 선택되어 왔습니다. 때문에 안정성과 내구성 측면에서 최상의 소재로 꼽힙니다. 추가로 오리지널징크는 화학적으로 안정하고 강철에 비해 열전도율이 낮아 단열성능이 좋다는 특징도 갖고 있습니다. 또한 튕김과 탄력성이 뛰어나 시공성이 편리하며 방수 기능을 제공하여 건축물의 내부를 보호하는 역할도 합니다. 이러한 이유로 오리지널징크는 건축 및 지붕 재질로 탁월한 선택지로 인정받고 있습니다. 만약 더 자세한 정보를 원신다면 연락해서 알려드리겠습니다.


In [12]:
# pd.DataFrame({'질문':test_df['질문'],'답변':preds}).to_csv('KoSOLAR_preprocess_data_aug_checkpoint_7000_1(text).csv', index = False)
pd.DataFrame({'질문':test_df['질문'],'답변':preds}).to_csv('KoSOLAR_preprocess_data_aug_checkpoint_epoch5(text)_1.csv', index = False)

### submission

In [13]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [14]:
submit = pd.read_csv('../sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.030561,0.036624,0.010163,-0.002619,0.065750,0.015505,-0.015209,0.026874,0.026260,...,0.025612,-0.039113,-0.016483,-0.034749,-0.008184,0.011365,0.063914,0.030753,0.018805,0.015426
1,TEST_001,-0.018386,-0.016006,-0.014936,-0.005052,0.069192,-0.009107,0.024706,0.009872,-0.002869,...,-0.014348,0.000725,-0.012790,-0.027778,-0.025515,0.052851,-0.016649,-0.004438,-0.003369,0.034420
2,TEST_002,0.016436,-0.045215,-0.032157,-0.006849,0.111231,-0.047366,-0.009675,-0.028557,0.047452,...,-0.006564,-0.012411,0.051366,-0.025075,0.009095,0.011781,-0.045464,-0.037856,-0.059106,0.096087
3,TEST_003,-0.015110,0.002423,-0.000310,0.021942,0.080822,-0.041524,-0.020717,0.026253,-0.000324,...,0.007246,-0.042186,0.095660,-0.041902,0.021512,0.046870,-0.026170,-0.072637,-0.020896,0.036715
4,TEST_004,0.009730,-0.044756,-0.036696,-0.005550,0.103400,-0.022103,0.046195,0.054985,0.003250,...,0.028013,-0.038538,0.060676,0.015277,-0.048877,0.000185,0.016453,0.032642,-0.013634,0.085767


In [15]:
# submit.to_csv('KoSOLAR_preprocess_data_aug_checkpoint_7000_1.csv', index = False)
# pd.read_csv('KoSOLAR_preprocess_data_aug_checkpoint_7000_1.csv')
submit.to_csv('KoSOLAR_preprocess_data_aug_checkpoint_epoch5_1.csv', index = False)
pd.read_csv('KoSOLAR_preprocess_data_aug_checkpoint_epoch5_1.csv')

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.030561,0.036624,0.010163,-0.002619,0.065750,0.015505,-0.015209,0.026874,0.026260,...,0.025612,-0.039113,-0.016483,-0.034749,-0.008184,0.011365,0.063914,0.030753,0.018805,0.015426
1,TEST_001,-0.018386,-0.016006,-0.014936,-0.005052,0.069192,-0.009107,0.024706,0.009872,-0.002869,...,-0.014348,0.000725,-0.012790,-0.027778,-0.025515,0.052851,-0.016649,-0.004438,-0.003369,0.034420
2,TEST_002,0.016436,-0.045215,-0.032157,-0.006849,0.111231,-0.047366,-0.009675,-0.028557,0.047452,...,-0.006564,-0.012411,0.051366,-0.025075,0.009095,0.011781,-0.045464,-0.037856,-0.059106,0.096087
3,TEST_003,-0.015110,0.002423,-0.000310,0.021942,0.080822,-0.041524,-0.020717,0.026253,-0.000324,...,0.007246,-0.042186,0.095660,-0.041902,0.021512,0.046870,-0.026170,-0.072637,-0.020896,0.036715
4,TEST_004,0.009730,-0.044756,-0.036696,-0.005550,0.103400,-0.022103,0.046195,0.054985,0.003250,...,0.028013,-0.038538,0.060676,0.015277,-0.048877,0.000185,0.016453,0.032642,-0.013634,0.085767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.021186,-0.051526,0.011415,0.014778,0.108633,-0.053231,0.063132,0.043898,0.039262,...,0.024669,0.026377,0.055006,-0.022823,0.012179,0.039910,0.037603,-0.026775,-0.019759,0.015620
126,TEST_126,0.028912,-0.067313,-0.023925,-0.007126,0.097756,-0.029806,0.018513,-0.002290,0.024025,...,-0.030745,-0.024280,-0.002310,-0.023282,-0.001842,0.023228,-0.006283,-0.048538,-0.002155,0.079428
127,TEST_127,-0.042239,-0.002520,-0.032053,0.044199,0.113132,-0.022009,-0.008353,0.004917,-0.061299,...,-0.036086,-0.111912,0.017338,-0.011919,-0.053069,0.042876,0.014790,-0.002259,-0.021569,0.026508
128,TEST_128,-0.013538,-0.029529,-0.063622,0.015755,0.104233,-0.022389,0.003241,0.035980,0.004411,...,0.020127,-0.003836,0.014070,-0.010673,-0.035780,0.014116,-0.006892,0.019590,0.029045,0.052772
